### Setup

In [2]:
!git clone https://github.com/britt-regan/health-plan-comparison.git

Cloning into 'health-plan-comparison'...
remote: Enumerating objects: 2743, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 2743 (delta 25), reused 40 (delta 14), pack-reused 2685
Receiving objects: 100% (2743/2743), 1.89 GiB | 24.78 MiB/s, done.
Resolving deltas: 100% (1337/1337), done.
Updating files: 100% (1806/1806), done.


In [3]:
import os
os.chdir('/content/health-plan-comparison/')

In [4]:
!pip install poetry
!poetry config virtualenvs.in-project true
!poetry install --no-root --no-ansi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.6/306.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 36.3 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.0.0
    Uninstalling platformdirs-4.0.0:
      Successfully uninstalled platformdirs-4.0.0
  Attempting uninstall: keyring
    Found existing installation: keyring 23.5.0
    Uninstal

In [5]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 50.3 MB/s eta 0:00:00


### Import Libraries

In [6]:
import fitz
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import pickle
import string

from nltk.corpus import stopwords
from transformers import pipeline, BartTokenizer, BartForConditionalGeneration

### Load Data

In [7]:
directory = os.listdir('2023 Competitor Docs')
data = []

for state_folder in directory:
    state_path = '2023 Competitor Docs/' + state_folder
    state_directory = os.listdir(state_path)

    for file in state_directory:
        file_path = state_path + '/' + file
        pdf_file = fitz.open(file_path)

        text = ''

        for page in pdf_file:
            extracted_text = page.get_text()
            text += extracted_text + '\n\n'

        data.append([state_folder, file, text])

documents = pd.DataFrame(data, columns = ['state', 'file', 'text'])

In [8]:
benefits_grid = pd.read_csv('Benefits Grid.csv')

benefits_grid = benefits_grid.rename(columns = {'CONTRACT_PLAN' : 'contract_plan',
                                                'County' : 'county',
                                                'Provider' : 'provider',
                                                'Annual Max ' : 'annual_max',
                                                'Implant Coverage (Y/N)' : 'implant_coverage',
                                                'Root Canal Coverage (Y/N)' : 'root_canal',
                                                'Healthy Food Rollover' : 'healthy_food_rollover',
                                                'OTC Rollover (Y/N)' : 'otc_rollover'})

benefits_grid['contract_plan'] = benefits_grid['contract_plan'].str.replace(' ', '')
benefits_grid['contract_plan'] = benefits_grid['contract_plan'].str.replace('-', '')
benefits_grid['contract_plan'] = benefits_grid['contract_plan'].str.replace('_', '')

benefits_grid = benefits_grid[['county',
                               'provider',
                               'contract_plan',
                               'annual_max',
                               'implant_coverage',
                               'root_canal',
                               'healthy_food_rollover',
                               'otc_rollover']]

benefits_grid = benefits_grid.replace(to_replace = ['Y?',
                                                    'Y (rider)',
                                                    'Y (allowance)',
                                                    'Y (most expensive rider)',
                                                    'Y (only with optional)',
                                                    'Y (only if rider is purchased)',
                                                    'with rider?',
                                                    'Y (only optional plans)',
                                                    'Y (with optional buy-in)',
                                                    'Y ',
                                                    'Y -- carries over each month and expires at the end of the year',
                                                    'Y -- $20 monthly allowance rolls over to next month and expires at the end of the year',
                                                    'Y -- $35 monthly allowance rolls over each month and expires at the end of the year',
                                                    'Y -- $30 monthly allowance rolls over each month and expires at the end of the year',
                                                    'Y, one month will carry over to the next month only within the same calendar quarter',
                                                    'Y  '],
                                      value = 'Y')

benefits_grid = benefits_grid.replace(to_replace = ['N?',
                                                    'N ',
                                                    'Unknown ',
                                                    '?',
                                                    'Unknown',
                                                    'UNK',
                                                    'N  ',
                                                    'UNK -- no mention in EOC',
                                                    'N -- note that EOC states that exceptions may apply.',
                                                    'NC'],
                                      value = 'N')

benefits_grid.fillna('N', inplace = True)

In [9]:
plan_list = benefits_grid[benefits_grid['contract_plan'] != 'EOC']['contract_plan']

documents['file'] = documents['file'].str.replace(' ', '')
documents['file'] = documents['file'].str.replace('-', '')
documents['file'] = documents['file'].str.replace('_', '')

documents['text_contract_plan'] = documents['text'].str.replace(' ', '')
documents['text_contract_plan'] = documents['text_contract_plan'].str.replace('-', '')
documents['text_contract_plan'] = documents['text_contract_plan'].str.replace('_', '')

In [10]:
def find_contract_plan(file_name):
    for plan in plan_list:
        if plan in file_name:
            return plan

documents['contract_plan_file'] = documents.apply(lambda row : find_contract_plan(row['file']), axis = 1)
documents['contract_plan_text'] = documents.apply(lambda row : find_contract_plan(row['text_contract_plan']), axis = 1)
documents['contract_plan'] = documents.contract_plan_file.combine_first(documents.contract_plan_text)
documents = documents.drop(['contract_plan_text', 'contract_plan_file'], axis = 1)

In [11]:
dataset = pd.merge(benefits_grid, documents, how = 'inner', on = ['contract_plan'])
dataset = dataset.drop(['file', 'text_contract_plan'], axis = 1)
dataset = dataset.drop_duplicates()

### BART

In [12]:
example = dataset['text'][0]
example

'Evidence of \nCoverage 2023\nUnitedHealthcare Dual Complete® (HMO-POS D-SNP)\nToll-free 1-866-480-1086, TTY 711\n8 a.m.-8 p.m. local time, 7 days a week\nmyUHCMedicare.com\nY0066_EOC_H0432_009_000_2023_C\n\n\nOMB Approval 0938-1051 (Expires: February 29, 2024)\nJanuary 1 – December 31, 2023\nEvidence of Coverage\nYour Medicare Health Benefits and Services and Prescription Drug \nCoverage as a Member of our plan\nThis document gives you the details about your Medicare health care and prescription drug \ncoverage from  January 1 – December 31, 2023. \n \nThis is an important legal document. Please keep it in a safe place.\nFor questions about this document, please contact Customer Service at \n1-866-480-1086. (TTY users should call 711). Hours are 8 a.m.-8 p.m. local time, \n7 days a week.\n \n This plan, UnitedHealthcare Dual Complete® (HMO-POS D-SNP), is insured through \nUnitedHealthcare Insurance Company or one of its affiliates. (When this Evidence of Coverage \nsays “we,” “us,” or

In [13]:
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# generate chunks of text \ sentences <= 1024 tokens
# generate chunks of text \ sentences <= 1024 tokens
def nest_sentences(document):
  nested = []
  sent = []
  length = 0
  for sentence in nltk.sent_tokenize(document):
    length += len(sentence)
    if length < 1024:
      sent.append(sentence)
    else:
      nested.append(sent)
      sent = [sentence]
      length = len(sentence)

  if sent:
    nested.append(sent)
  return nested

# generate summary on text with <= 1024 tokens
def generate_summary(nested_sentences):
  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.to(device).generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
generate_summary(example)